In [1]:
!pip install -q -U torch --index-url https://download.pytorch.org/whl/cu117
!pip install -q -U -i https://pypi.org/simple/ bitsandbytes
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl
!pip install -q -U peft

In [2]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import Dataset
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/kaggle/input/drugs-side-effects-and-medical-condition/drugs_side_effects_drugs_com.csv')

In [ ]:
def create_input(row):
    medical_info = (
        f"Patient Info:\n"
        f"- Medical Condition: {row['medical_condition']}\n"
    )
    return medical_info

In [ ]:
def create_output(row):
    output = (
        f"Considering your medical condition of {row['medical_condition']}, "
        f"you might want to take drugs like {row['drug_name']}. "
        f"This might accompany side effects such as {row['side_effects']}, so you should be aware of this when taking it. "
    )
    return output


In [ ]:
df['input'] = df.apply(create_input, axis=1)
df['output'] = df.apply(create_output, axis=1)

In [ ]:
df['text'] = df.apply(lambda row: f"input: {row['input']}\noutput: {row['output']}", axis=1)

In [ ]:
df.dropna(subset=['input', 'output', 'text'], inplace=True)
df = df.sample(frac=1).reset_index(drop=True)


In [ ]:
example_row = df.iloc[0]
print("Input:")
print(example_row['input'])
print("\nOutput:")
print(example_row['output'])

Input:
Patient Info:
- Medical Condition: Acne


Output:
Considering your medical condition of Acne, you might want to take drugs like ethinyl estradiol / norethindrone. This might accompany side effects such as hives ; difficult breathing; swelling of your face, lips, tongue, or throat. This medicine may cause serious side effects. Stop using this medicine and call your doctor at once if you have: heart attack symptoms--chest pain or pressure, pain spreading to your jaw or shoulder, nausea, sweating; increased blood pressure--severe headache , blurred vision, pounding in your neck or ears, anxiety , nosebleed; signs of a stroke--sudden numbness or weakness (especially on one side of the body), sudden severe headache, slurred speech, problems with vision or balance; signs of a blood clot--sudden vision loss, stabbing chest pain, feeling short of breath, coughing up blood , pain or warmth in one or both legs; swelling or tenderness in your stomach; jaundice (yellowing of the skin or eye

In [ ]:
train_data = Dataset.from_pandas(df[['text']])

In [ ]:
model_name = "/kaggle/input/gemma/transformers/2b-it/3"
compute_dtype = getattr(torch, "float16")

def load_model_and_tokenizer():
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=compute_dtype,
        low_cpu_mem_usage=True,
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name, max_seq_length=1024)
    return model, tokenizer

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, tokenizer = load_model_and_tokenizer()
model.to(device)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-

In [ ]:
def get_medicine_recommendation(medical_info, model, tokenizer):
    matching_rows = df[df['medical_condition'].str.lower() == medical_info.lower()]
    if matching_rows.empty:
        return f"we do not have information about this drug."
    row = matching_rows.iloc[0]
    return create_output(row)  

In [ ]:
example_input = df.iloc[340]['input']
medical_condition = example_input.split(':')[-1].strip() 

print("What drug is recommended for my medical condition?")
print(f"Medical condition : {medical_condition}")
print("\nGenerated Advice:")
print(get_medicine_recommendation(medical_condition, model, tokenizer))

What drug is recommended for my medical condition?
Medical condition : Angina

Generated Advice:
Considering your medical condition of Angina, you might want to take drugs like nadolol. This might accompany side effects such as hives ; difficulty breathing; swelling of your face, lips, tongue, or throat. Nadolol may cause serious side effects. Call your doctor at once if you have: a light-headed feeling, like you might pass out; slow heartbeats; shortness of breath (even with mild exertion), swelling, rapid weight gain; or bronchospasm (wheezing, chest tightness, trouble breathing). Common side effects of nadolol may include: numbness or cold feeling in your hands or feet; dizziness; feeling tired; upset stomach , vomiting , diarrhea , constipation ; vision problems; or mood changes, confusion, memory problems., so you should be aware of this when taking it. 
